In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
%matplotlib inline

## Importing Data and Cleaning

In [4]:
data = pd.read_csv("main.csv")

In [5]:
len(data)

88620

In [6]:
data.index = data["id"]

In [7]:
data = data.drop(columns=["id"])

In [8]:
data.head()

,skill_set,subcategory
id,,
0,"('consulting', 'advising', 'career-growth')",Self Improvement\r\n
1,"('consulting', 'advising', 'career-growth')",Create Business & Startup\r\n
2,"('consulting', 'advising', 'career-growth')",Finance & Accounting\r\n
3,"('consulting', 'advising', 'career-growth')",Company’s Trade Registration \r\n
4,"('consulting', 'advising', 'career-growth')",Marketing\r\n


In [9]:
data['subcategory'] = data["subcategory"].apply(lambda x: x.strip())

In [10]:
data.head()

,skill_set,subcategory
id,,
0,"('consulting', 'advising', 'career-growth')",Self Improvement
1,"('consulting', 'advising', 'career-growth')",Create Business & Startup
2,"('consulting', 'advising', 'career-growth')",Finance & Accounting
3,"('consulting', 'advising', 'career-growth')",Company’s Trade Registration
4,"('consulting', 'advising', 'career-growth')",Marketing


In [14]:
data_2 = data.head(10000)

## kNN Algorithm

In [15]:
len(data_2)

10000

In [16]:
pd.get_dummies(data_2["skill_set"]).head() # Convert type to categorical values for ease

,"('adobe-after-effect', 'adobe-illustrator', 'adobe-muse')","('adobe-after-effect', 'adobe-illustrator', 'adobe-photo')","('adobe-after-effect', 'adobe-illustrator', 'adobe-premiere-pro')","('adobe-after-effect', 'adobe-illustrator', 'adobe-xd')","('adobe-after-effect', 'adobe-illustrator', 'final-cut-pro')","('adobe-after-effect', 'adobe-illustrator', 'ui')","('adobe-after-effect', 'adobe-illustrator', 'ux')","('adobe-after-effect', 'adobe-muse', 'adobe-illustrator')","('adobe-after-effect', 'adobe-muse', 'adobe-photo')","('adobe-after-effect', 'adobe-muse', 'adobe-premiere-pro')",...,"('valuation', 'investment-banking', 'consulting')","('valuation', 'investment-banking', 'financial-modeling')","('valuation', 'investment-banking', 'startup')","('valuation', 'startup', 'advising')","('valuation', 'startup', 'business-valuation')","('valuation', 'startup', 'career-growth')","('valuation', 'startup', 'consulting')","('valuation', 'startup', 'financial-modeling')","('valuation', 'startup', 'investment-banking')",skill_set
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
data_features = pd.concat([pd.get_dummies(data_2[["subcategory"]]), pd.get_dummies(data_2[["skill_set"]])], axis=1)

In [22]:
data_features.head()

,subcategory_3D & Perspective,subcategory_Advertising Banner,subcategory_Branding,subcategory_CAD Drawing,subcategory_Character Design,subcategory_Company’s Trade Registration,subcategory_Create Business & Startup,subcategory_Dicut & Photoshop,subcategory_Finance & Accounting,subcategory_Illustration,...,"skill_set_('valuation', 'investment-banking', 'consulting')","skill_set_('valuation', 'investment-banking', 'financial-modeling')","skill_set_('valuation', 'investment-banking', 'startup')","skill_set_('valuation', 'startup', 'advising')","skill_set_('valuation', 'startup', 'business-valuation')","skill_set_('valuation', 'startup', 'career-growth')","skill_set_('valuation', 'startup', 'consulting')","skill_set_('valuation', 'startup', 'financial-modeling')","skill_set_('valuation', 'startup', 'investment-banking')",skill_set_skill_set
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
from sklearn.preprocessing import MaxAbsScaler

max_scaler = MaxAbsScaler()
data_features = max_scaler.fit_transform(data_features)

In [24]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree').fit(data_features)
distances, indices = neigh.kneighbors(data_features)

In [26]:
# Need this function to call our main function if we get a name
def idtoindex(id):
    return data_2[data_2["id"] == id].index.tolist()[0]

In [44]:
# This function takes in either name or ID and can spit out some 'similar'

def actualsimilar(idd):
    for i in indices[idd][1:]: # Removing the first one because it is itself
        print i#data_2.ix[i]["skill_set"]

def similarusers(idd = None):
    return actualsimilar(idd)

In [46]:
similarusers(idd = 79)

81
1451
83
77
